In [92]:
import pandas as pd

In [56]:
df = pd.read_csv("train.txt", sep=";")

In [57]:
df.head()

,i didnt feel humiliated,sadness
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [58]:
df.columns = ['sentence', 'emotion']

In [59]:
df.head()

,sentence,emotion
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [61]:
category = pd.get_dummies(df.emotion)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='emotion')
df_baru

,sentence,anger,fear,joy,love,sadness,surprise
0,i can go from feeling so hopeless to so damned...,0,0,0,0,1,0
1,im grabbing a minute to post i feel greedy wrong,1,0,0,0,0,0
2,i am ever feeling nostalgic about the fireplac...,0,0,0,1,0,0
3,i am feeling grouchy,1,0,0,0,0,0
4,ive been feeling a little burdened lately wasn...,0,0,0,0,1,0
...,...,...,...,...,...,...,...
15994,i just had a very brief time in the beanbag an...,0,0,0,0,1,0
15995,i am now turning and i feel pathetic that i am...,0,0,0,0,1,0
15996,i feel strong and good overall,0,0,1,0,0,0
15997,i feel like this was such a rude comment and i...,1,0,0,0,0,0


In [81]:
komentar = df_baru['sentence'].values
emosi = df_baru[['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']].values

In [74]:
from sklearn.model_selection import train_test_split
komentar_latih, komentar_test, emosi_latih, emosi_test = train_test_split(komentar, emosi, test_size=0.2)

In [75]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [76]:
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(komentar_latih) 
tokenizer.fit_on_texts(komentar_test)

In [77]:
sekuens_latih = tokenizer.texts_to_sequences(komentar_latih)
sekuens_test = tokenizer.texts_to_sequences(komentar_test)

In [78]:
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [85]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')

])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [91]:
num_epochs = 10
class myCallback (tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.80):
      print('\nAkurasi telah mencapai >80%!')
      self.model.stop_training=True

callbacks = myCallback()
history = model.fit(padded_latih, emosi_latih, epochs=num_epochs, 
                    validation_data=(padded_test, emosi_test), verbose=2)

Epoch 1/10
400/400 - 14s - loss: 0.0203 - accuracy: 0.9935 - val_loss: 0.5754 - val_accuracy: 0.9013
Epoch 2/10
400/400 - 14s - loss: 0.0096 - accuracy: 0.9965 - val_loss: 0.6607 - val_accuracy: 0.8947
Epoch 3/10
400/400 - 14s - loss: 0.0112 - accuracy: 0.9955 - val_loss: 0.5849 - val_accuracy: 0.9044
Epoch 4/10
400/400 - 14s - loss: 0.0074 - accuracy: 0.9972 - val_loss: 0.6519 - val_accuracy: 0.8997
Epoch 5/10
400/400 - 14s - loss: 0.0104 - accuracy: 0.9964 - val_loss: 0.6385 - val_accuracy: 0.9031
Epoch 6/10
400/400 - 14s - loss: 0.0141 - accuracy: 0.9950 - val_loss: 0.6344 - val_accuracy: 0.8972
Epoch 7/10
400/400 - 14s - loss: 0.0143 - accuracy: 0.9953 - val_loss: 0.6292 - val_accuracy: 0.9038
Epoch 8/10
400/400 - 14s - loss: 0.0064 - accuracy: 0.9969 - val_loss: 0.7089 - val_accuracy: 0.9006
Epoch 9/10
400/400 - 14s - loss: 0.0059 - accuracy: 0.9970 - val_loss: 0.8112 - val_accuracy: 0.8941
Epoch 10/10
400/400 - 14s - loss: 0.0223 - accuracy: 0.9931 - val_loss: 0.5636 - val_accura